# Подготовка корпуса текстов 17-го века для AIJ 2020 PG

In [147]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import string
import re

## Формируем корпус для обучения модели из коллекции текстов 17 века, которая была предложена организаторами соревнования GramEval2020

In [148]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [149]:
letters = [' ', ')', '+', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '[', ']', 'i', 'k', 'l', '|', '×', 'ǂ',
           'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х',
           'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'і', 'ѣ', '–', '…', '⊕', '⊗']

In [150]:
exc_sym = '!"#$%&\'(*,-.:;<=>?@\\^_`{}~'

In [151]:
grameval = Path("grameval_17_century.txt")
if not grameval.is_file():
    with open("grameval_17_century.txt", "w", encoding='utf-8') as fout:   #, errors='ignore'
        with open("GramEval2020-17cent-dirty.conllu", "r", encoding='utf-8') as fin: #, errors='ignore'
            sent = ""
            cnt = 0
            for line in tqdm(fin):
                cnt += 1
                if len(line) == 1:
                    sent = sent.strip()
                    sent = re.sub("\n", "", sent)
                    if (len(sent) > 3):
                        fout.write(sent + "\n")
                    sent = ""
                else:
                    if line.startswith("#"):
                        continue
                    line = line.split("\t")[1]
                    line = line.lower()
                    if line.startswith("{"):
                        continue
                    if line.startswith("<"):
                        continue
    #                 if line in exc_sym:
    #                     continue
                    sent += ''.join([ch if ch in letters else " " for ch in line])
                    sent = sent.strip()
    #                 sent += line
                    sent += " "
                    if len(sent) > 65:
                        sent = re.sub("\n", "", sent)
                        fout.write(sent + "\n")
                        sent = ""

3299994it [00:47, 69076.94it/s]


In [152]:
sent = " "
sent = sent.strip()
len(sent)

0

## Формируем модель

In [153]:
fname = "grameval_17_century.txt"

In [154]:
def read_grameval(fname=fname):
    with open(fname, "r", encoding='utf-8') as f:
        lines = [x[:-1] for x in f.readlines()]
        return lines

In [155]:
grameval_texts = read_grameval(fname)

grameval_texts[-5]

'и съ тое земли по прежнимъ книгамъ по окладнымъ ясашнымъ книгамъ денги '

In [156]:
grameval_texts[-3]

'къ сей владѣной выписи великого государя царя и великого князя алексѣя '

In [157]:
len(grameval_texts)

284622

In [158]:
grameval_texts[20060]

'федоровичю всеа русии из арзамаса стольник и воевода князь михайло '

## Зададим набор правил для аугментации данных с использованием шума и специфики стиля Петра I

In [159]:
import random

# p of substitution = 1/znam
znam = 2

rules = []

#Традиционно над строкой Петр пишет «з» и «с», конечное «х», также «к» перед широкой размашистой «ж»
rules.append(('з',''))
rules.append(('c',''))
rules.append(('x',''))
rules.append(('кж', 'ж'))
#вместо старого «ѧ» уже регулярно употребляет вполне современное «я»
rules.append(('ѧ', 'я'))
#Не любит буквы «s» («зело») и «ѵ» 
rules.append(('s', ''))
rules.append(('ѵ', ''))
#Мягкий знак пропускает
rules.append(('ь', ''))


def replace_letters(line, to_replace, replace_by, znam=2):
    new_line = ''
    for letter in line:
        if letter == to_replace and random.randint(0, znam - 1) % znam == 0:
            new_line = new_line + replace_by
        else:
            new_line = new_line + letter
    return new_line
    

def apply_rule(lines, rule, znam=2):
    to_replace, replace_by = rule
    res = [
        replace_letters(line, to_replace, replace_by) for line in lines
    ]
    return res

In [160]:
print("Peter's writing rules:")
print(rules)

Peter's writing rules:
[('з', ''), ('c', ''), ('x', ''), ('кж', 'ж'), ('ѧ', 'я'), ('s', ''), ('ѵ', ''), ('ь', '')]


In [161]:
for rule in tqdm(rules, total=len(rules), desc="Generating data..."):
    grameval_texts = apply_rule(grameval_texts, rule)

Generating data...: 100%|████████████████████████| 8/8 [00:40<00:00,  5.03s/it]


In [162]:
grameval_texts[-1]

'приложилъ столникъ и воевода алексѣй петровичъ головинъ'

In [163]:
len(grameval_texts)

284622

## Построим словарь для генерации шума

In [164]:
import os
from collections import Counter


trans_dir = 'C:\\tmp\\datasets\\train\\words'
image_dir = 'C:\\tmp\\datasets\\train\\images'

english = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'm', 'n' ,'o', 'p', 'r', 's', 't', 'u', 'w']


def process_texts(image_dir, trans_dir):
    lens = []
    include_english = 0
    letters = ''

    lines = []
    names = []
    
    all_files = os.listdir(trans_dir)
    for filename in os.listdir(image_dir):
        if filename[:-3]+'txt' in all_files:
            name, ext = os.path.splitext(filename)
            txt_filepath = os.path.join(trans_dir, name + '.txt')
            with open(txt_filepath, 'r', encoding='utf-8') as file:
                data = file.read()
                if len(data)==0:
                    continue
                if len(set(data).intersection(english))>0:
                    continue

                lines.append(data)
                names.append(filename)
                lens.append(len(data))
                letters += data

    print('Максимальная длина строки:', max(lens))
    print('Количество строк с английскими буквами ', include_english)

    return names, lines, Counter(letters)

In [165]:
names, lines, cnt = process_texts(image_dir,trans_dir)

Максимальная длина строки: 71
Количество строк с английскими буквами  0


In [166]:
letters = sorted(list(cnt.keys()))
print('Символы train:', ' '.join(letters))

Символы train:   ) + / 0 1 2 3 4 5 6 7 8 9 [ ] i k l | × ǂ а б в г д е ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я і ѣ – … ⊕ ⊗


## Аугментация
- добавление шума из словаря на посимвольном уровне;
- удаление пробелов с определенной вероятностью.

In [167]:
df = pd.DataFrame(lines + grameval_texts, columns=["trg"])

In [168]:
def add_noise(text, symbols=letters, znam=4):
    text = list(text)
    num = len(text) // znam
    indexes = random.sample(range(0, len(text)), num)
    for i in indexes:
        if text[i]!=' ':
            text[i] = random.choice(symbols)
        else:
            del_space = np.random.choice([True, False], p=[0.3, 0.7])
            if del_space:
                text[i] = text[i].replace(" ", "")
    return ''.join(text)


df["src"] = [add_noise(t, znam=4) for t in df["trg"].tolist()]

In [169]:
df.head()

,trg,src
0,iз питербурха въ 14 д апъ петръ,iз п612рбурха въ 14д апъ пежръ
1,рѣля 1712,пѣля 171е
2,войдут в полшу то i нам волно i тогда паки,вой/ут l полшу тс i нам волн× ц 5огда пакѣ
3,прежнее разорение будет в полше того для лут,врежнец ]а8орениш яждет в полше тюко д[я луч
4,че iм малым сим удоволствоват нас нынѣ,ч/ iм малым сим мдоiолств]ѣат нас н…3ѣ


In [170]:
df["id"] = np.arange(df.shape[0])

In [171]:
df["cn"] = [len(ln) for ln in df["trg"].values]

In [172]:
df.head()

,trg,src,id,cn
0,iз питербурха въ 14 д апъ петръ,iз п612рбурха въ 14д апъ пежръ,0,31
1,рѣля 1712,пѣля 171е,1,9
2,войдут в полшу то i нам волно i тогда паки,вой/ут l полшу тс i нам волн× ц 5огда пакѣ,2,42
3,прежнее разорение будет в полше того для лут,врежнец ]а8орениш яждет в полше тюко д[я луч,3,44
4,че iм малым сим удоволствоват нас нынѣ,ч/ iм малым сим мдоiолств]ѣат нас н…3ѣ,4,38


In [173]:
col_names_list = df.columns.tolist()
col_names_list

['trg', 'src', 'id', 'cn']

In [174]:
df = df[[col_names_list[2], col_names_list[1], col_names_list[0], col_names_list[3]]]
df.head()

,id,src,trg,cn
0,0,iз п612рбурха въ 14д апъ пежръ,iз питербурха въ 14 д апъ петръ,31
1,1,пѣля 171е,рѣля 1712,9
2,2,вой/ут l полшу тс i нам волн× ц 5огда пакѣ,войдут в полшу то i нам волно i тогда паки,42
3,3,врежнец ]а8орениш яждет в полше тюко д[я луч,прежнее разорение будет в полше того для лут,44
4,4,ч/ iм малым сим мдоiолств]ѣат нас н…3ѣ,че iм малым сим удоволствоват нас нынѣ,38


In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290727 entries, 0 to 290726
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      290727 non-null  int32 
 1   src     290727 non-null  object
 2   trg     290727 non-null  object
 3   cn      290727 non-null  int64 
dtypes: int32(1), int64(1), object(2)
memory usage: 7.8+ MB


In [176]:
df.loc[df["cn"] == 0]

,id,src,trg,cn


In [177]:
df.loc[df["cn"] > 81]

,id,src,trg,cn
13465,13465,грамота ие–а пютріарйа 110 го г1дн что днн0 вk...,грамота иева патриарха 110 го году что дана во...,87
27522,27522,и чокцоняемыщ го трііцф во единоиъ бзжествѣ те...,и поклоняемый во троицы во единомъ божествѣ те...,82
33649,33649,дарофѣй егмже крѣпное житие сазч мрмымцритъ iщ...,дорофѣй егоже крѣпкое житие самъ архимаритъ ещ...,82
33803,33803,ǂожига9⊗и ⊗ всякими кkвеынамиосв]рняеми паче ж...,сожигаеми и всякими сквернами осверняеми паче ...,82
36604,36604,а х4тѣни– свожму ласканичм пшево[водя оѣ1ты 3м...,к хотѣнию своему ласканием превозводя обѣты им...,82
37221,37221,ф2кавщн ащь и образомэ к0×крнымъ яв5яшежя к не...,лукавыя аще и образомъ коварнымъ являшеся к не...,84
37573,37573,итоли6ц множх8тв9пл[iкихъ людей і лиǂовъ]ких и...,и толико множество полскихъ людей и литовъских...,83
39532,39532,где ув бвой делъсь что сицеуничижатб–н⊗нпоіедо...,где ум твой делъся что сице уничижателно о поб...,82
53537,53537,а ыаъ ѣтолба и отф ямы до ямской слкбоды п8пр/...,а отъ столба и отъ ямы до ямской слободы напра...,84
56783,56783,его вохяошаем5а объпти сши сл/шомъ тобромыслен...,его вопрошаемое объяти сши словомъ добромыслен...,82


In [178]:
df.to_csv("augmented_data.csv", index=False)

## Используем заранее аугментированные данные, где:
- id – id семпла;
- src – исходная последовательность;
- trg – целевая последовательность;
- cn – длина исходной последовательности в символах.

Датафрейм включает в себя аугментацию предиктов бейзлайна на валидационном сете (baseline.ipynb) и предложений из grameval_17_century.txt.